# 1. Import Dependencies

In [1]:
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback

# 2. Data Preparation

In [2]:
df = pd.read_csv('push_up_up_down.csv')

In [3]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,0,0.327129,0.864504,0.046474,0.999743,0.318453,0.856404,0.023865,0.999770,0.318511,...,0.092763,0.590589,0.693436,0.905446,-0.180688,0.949445,0.685352,0.895096,0.053819,0.606479
1,1,0.332821,0.698585,-0.069984,0.999111,0.325864,0.687071,-0.092433,0.999270,0.325865,...,0.239815,0.721270,0.684267,0.906728,-0.078966,0.949122,0.680650,0.888815,0.180140,0.722509
2,1,0.659933,0.686185,-0.297425,0.999456,0.670619,0.681447,-0.313336,0.999551,0.672110,...,-0.117281,0.844597,0.436931,0.834366,-0.070264,0.941264,0.288310,0.828274,-0.212475,0.847874
3,1,0.650864,0.686589,-0.335529,0.999490,0.661480,0.683025,-0.345861,0.999582,0.663704,...,-0.121955,0.851118,0.410835,0.838893,-0.056105,0.935920,0.288956,0.837479,-0.232930,0.856104
4,1,0.662774,0.678150,-0.444944,0.999518,0.672196,0.673234,-0.458909,0.999614,0.674197,...,-0.153514,0.862419,0.477668,0.779791,0.095701,0.930550,0.291468,0.840468,-0.255697,0.867524


In [76]:
X = df.drop('class', axis=1)
y = df['class']

In [77]:
# X_min = X.min(axis=0)
# X_max = X.max(axis=0)
# normalized_X = (X - X_min) / (X_max - X_min)


In [78]:
# normalized_X.min()

In [79]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 3. Modelling

In [80]:
model = tf.keras.Sequential([
    layers.Dense(64, input_dim=132, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1, activation='sigmoid')  
])


In [81]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [82]:
class AccuracyStopCallback(Callback):
    def __init__(self, target_accuracy=0.97):
        super(AccuracyStopCallback, self).__init__()
        self.target_accuracy = target_accuracy
        self.val_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= self.target_accuracy and logs.get('val_accuracy') >= self.target_accuracy:
            print(f"\nReached target accuracy of {self.target_accuracy}, stopping training!")
            self.model.stop_training = True

In [83]:
callback = AccuracyStopCallback(target_accuracy=0.97)

In [84]:
model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2, callbacks=[callback])

Epoch 1/20
55/55 [==============================] - 1s 13ms/step - loss: 1.4807 - accuracy: 0.9335 - val_loss: 1.0118 - val_accuracy: 1.0000
Epoch 2/20
51/55 [==========================>...] - ETA: 0s - loss: 0.7851 - accuracy: 0.9975
Reached target accuracy of 0.97, stopping training!
55/55 [==============================] - 0s 9ms/step - loss: 0.7731 - accuracy: 0.9977 - val_loss: 0.5755 - val_accuracy: 1.0000


In [85]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model_up_down.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmp97lssf86\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmp97lssf86\assets


# 4. Testing

In [86]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils 
mp_pose = mp.solutions.pose

landmarks = ["class"]
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

vid_path = "test.mp4"
cap = cv2.VideoCapture(vid_path)
current_stage = ''

# Initiate Pose Model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # Streaming the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize the frame to the desired window size
        frame = cv2.resize(frame, (720, 600))

        # Recolor feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detections
        results = pose.process(image)

        # Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        try:
            if results.pose_landmarks:
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                row = np.expand_dims(row, axis=0)  
                # row_min = row.min(axis=1, keepdims=True)
                # row_max = row.max(axis=1, keepdims=True)
                # row_range = row_max - row_min
                
                # # Avoid division by zero by setting row_range to 1 where it is zero
                # row_range[row_range == 0] = 1
                # normalized_row = (row - row_min) / row_range
                prediction = model.predict(row)
                print(f'Result: {prediction[0][0]}')
                
                if prediction[0][0] > 0.5:
                    current_stage = 'Up'
                elif prediction[0][0] <= 0.5:
                    current_stage = 'Down'

                cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

                cv2.putText(image, 'CLASS', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(prediction[0][0], 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except Exception as e:
            print(f"Error: {e}")


        # Stream video result
        cv2.imshow("Raw Cam Feed", image)

        # Press 'q' to stop the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


Result: 0.9877142906188965
Result: 0.9883318543434143
Result: 0.9883925318717957
Result: 0.9894494414329529
Result: 0.9904332160949707
Result: 0.9904898405075073
Result: 0.990250825881958
Result: 0.9891605973243713
Result: 0.9878737330436707
Result: 0.9852932095527649
Result: 0.9844787120819092
Result: 0.9836059808731079
Result: 0.9714760184288025
Result: 0.9314356446266174
Result: 0.8777024745941162
Result: 0.8892108798027039
Result: 0.8315930366516113
Result: 0.6428140997886658
Result: 0.5084757208824158
Result: 0.3999424874782562
Result: 0.3320712149143219
Result: 0.2610797882080078
Result: 0.18900083005428314
Result: 0.13665316998958588
Result: 0.11077731847763062
Result: 0.08988571912050247
Result: 0.08829368650913239
Result: 0.05300256982445717
Result: 0.05675904452800751
Result: 0.053927090018987656
Result: 0.05195212736725807
Result: 0.04740048944950104
Result: 0.04485977068543434
Result: 0.051265958696603775
Result: 0.04541846364736557
Result: 0.05460197106003761
Result: 0.051